In [5]:
!pip install transformers datasets accelerate -q

In [9]:
pip install pyabsa


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/575.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/575.5 kB ? eta -:--:--
   ------------------ --------------------- 262.1/575.5 kB ? eta -:--:--
   ------------------ --------------------- 262.1/575.5 kB ? eta -:--:--
   ---------------------------------------- 575.5/575.5 kB 801.4 kB/s  0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16282 sha256=95243913d47479d28e8ce6041dffc60c99d6c7974f23db63b7eb788b325ffb8a
  Stored in directory: c:\users\apara\appdata\local\pip\cache\wheels\bc\92\f0\243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval

   ----------------------------------------  0/12 [xlsxwriter]
   ----------------------------------------  0/12 [xlsxwriter]
   ----------------------------------------  0/12 [xlsxwriter]
   ------------------------------

  DEPRECATION: Building 'seqeval' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'seqeval'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
tensorflow-intel 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.13.0 which is incompatible.


In [12]:
import os
os.environ["USE_TF"] = "0"
os.environ["USE_FLAX"] = "0"

from transformers import pipeline


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
from pyabsa import AspectTermExtraction as ATEPC
absa_pipeline = pipeline(
    "text-classification",
    model="yangheng/deberta-v3-base-absa-v1.1",
    return_all_scores=True
)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
Unable to convert function return value to a Python type! The signature was
	() -> handle

In [8]:
import pandas as pd
import os
import glob

In [ ]:
input_dir = r"D:\AIML\reddit mental health\data\processed\clean_csv"
output_dir = r"D:\AIML\reddit mental health\data\processed\absa_csv"
os.makedirs(output_dir, exist_ok=True)
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))

In [9]:
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)

        # Ensure body column exists and is string
        if 'body' not in df.columns:
            print(f"Skipping {file_path}: No 'body' column found.")
            continue
        df['body'] = df['body'].fillna("").astype(str)

        # Map comment IDs to their text
        id_to_text = dict(zip(df['id'], df['body']))

        # Create parent_text column by mapping parent_id to its text
        df['parent_text'] = df['parent_id'].map(id_to_text).fillna("").astype(str)

        # Combine context safely
        df['context_text'] = df.apply(
            lambda row: row['parent_text'] + " [SEP] " + row['body']
            if row['parent_text'] else row['body'],
            axis=1
        )

        output_path = os.path.join(output_dir, os.path.basename(file_path))
        df.to_csv(output_path, index=False)
        print(f"Processed and saved: {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ggfmyg.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ggjfl9.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ggnkgn.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1gh41pn.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1gh6wk4.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ghc9ti.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ghcfvs.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ghchxr.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1ghcjv5.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1gik043.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1gin7az.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1gjer2l.csv
Processed and saved: D:\AIML\reddit mentsl health\data\processed\1gjh58v.csv

In [10]:
from tqdm import tqdm
from transformers import pipeline

In [ ]:
# Fix the input directory path
input_dir = r"D:\AIML\reddit mental health\data\txtprocessed"
output_file = r"D:\AIML\reddit mental health\data\final_feature_sentiment.csv"

results = []
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))


In [17]:
def normalize_absa_output(output):
    """
    Normalize ABSA pipeline output into a flat list of dicts
    with keys: label, score
    """
    # If it's a single dict
    if isinstance(output, dict):
        return [output]

    # If it's already a list of dicts
    if all(isinstance(x, dict) for x in output):
        return output

    # If it's a nested list (list of lists of dicts)
    flat = []
    for item in output:
        if isinstance(item, list):
            flat.extend(normalize_absa_output(item))  # recursion
        elif isinstance(item, dict):
            flat.append(item)
    return flat


In [ ]:
# Process each file
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        
        # Process each row in the dataframe
        for _, row in tqdm(df.iterrows(), total=len(df), desc=os.path.basename(file_path)):
            try:
                text_input = str(row['context_text']).strip()
                if not text_input:
                    continue

                output = absa_pipeline(text_input)
                output = normalize_absa_output(output)  # ensure flat list

                for aspect in output:
                    if not isinstance(aspect, dict) or 'label' not in aspect:
                        continue  # skip malformed entries

                    label_parts = aspect['label'].split('#')
                    if len(label_parts) == 2 and aspect['score'] >= 0.6:
                        feature, sentiment = label_parts
                        results.append({
                            'post_id': row.get('post_id', None),
                            'comment_id': row.get('id', None),
                            'feature': feature.lower(),
                            'sentiment': sentiment,
                            'score': aspect['score'],
                            'context_text': text_input
                        })
            except Exception as e:
                print(f"Error processing row: {e}")
                
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

# Save the results
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

1ggfmyg.csv: 100%|██████████| 58/58 [00:24<00:00,  2.35it/s]
